In [ ]:
import numpy as np

def realign_features_all_lstm_output(inferred_phases_all, n_tracks, lstm_shift = 5):
    
   
    inferred_phases_all = inferred_phases_all.reshape(n_tracks, -1)
    n_timesteps = inferred_phases_all.shape[1]
    inferred_phases_all_shifted = np.zeros(n_tracks,n_timesteps+5)+np.nan

    inferred_phases_all_shifted[:, 5:] = inferred_phases_all #5 is lstm_shift
    return inferred_phases_all_shifted


def get_first_rev_latency(binary_mat, stim_onset, stim_offset):
    mat = np.nan_to_num(binary_mat, nan=0)  # Replace NaNs with 0

    # Rows where the value at stim_onset is 0
    valid_rows = mat[:, stim_onset] == 0

    # Slice the region to search
    search_region = mat[:, stim_onset+1:stim_offset+1]  # shape: (n_rows, time_window)

    # Create a mask for the first 1 per row (set invalid rows to False everywhere)
    first_one_mask = (search_region == 1) & valid_rows[:, None]

    # Find the first occurrence of 1 per row, -1 if not found
    first_one_idx = np.argmax(first_one_mask, axis=1)
    has_one = np.any(first_one_mask, axis=1)

    # Set invalid ones to -1
    latencies = np.where(has_one, first_one_idx + 1, -1)  # +1 to account for stim_onset+1 index offset

    return latencies


def get_transition_rows(binary_mat, stim_onset, stim_offset):
    # Ensure we are working with a boolean mask ignoring NaNs
    mat = np.nan_to_num(binary_mat, nan=0)  # Replace NaNs with 0 for comparison

    onset_zero = mat[:, stim_onset] == 0  # Check if value at stim_onset is 0
    becomes_one = np.any(mat[:, stim_onset+1:stim_offset+1] == 1, axis=1)  # Check for a 1 between onset and offset

    return np.where(onset_zero & becomes_one)[0]  # Return the indices of matching rows

n_tracks  = resampled_vel.shape[0]
rev_all = behaviors_all==1

rev_all = rev_all.reshape(n_tracks, -1)
n_timesteps = rev_all.shape[1]
rev_all_shifted = np.zeros(n_tracks,n_timesteps+5)+np.nan
# rev_all_shifted = realign_features_all_lstm_output(rev_all, n_tracks, lstm_shift = 5)

# n_tracks  = resampled_vel.shape[0]
inferred_phases_all = inferred_phases_all.reshape(n_tracks, -1)
n_timesteps = inferred_phases_all.shape[1]
inferred_phases_all_shifted = np.zeros(n_tracks,n_timesteps+5)+np.nan

inferred_phases_all_shifted[:, 5:] = inferred_phases_all

stim_to_latencies = {}
stim_to_phases = {}
#get revs at onset 
slow_fps = 1.6
stim_duration_s = 5
for stim_i, stim_onset in enumerate(onsets):
    stim_offset = stim_onset + stim_duration_s*slow_fps
    latencies = get_first_rev_latency(rev_all_shifted, stim_onset, stim_offset)
    responder_track_is = np.argwhere(latencies!=-1).flatten()
    stim_latencies = latencies[responder_track_is]
    stim_phases = inferred_phases_all_shifted[stim_onset, responder_track_is]
    
    stim_to_latencies[stim_i] = stim_latencies
    stim_to_phases[stim_i] = stim_phases
    
    fig,ax = plt.subplots()
    ax.scatter(stim_latencies, stim_phases)
    ax.set_title(f"{neuron}; {stim_i}")
    

In [ ]:


# inferred_phases_all_shifted = realign_features_all_lstm_output(inferred_phases_all, n_tracks, lstm_shift = 5)
def interpolate_features(features, original_fps=1.6, target_fps=6.0):
    n_frames, n_features = features.shape
    original_times = np.arange(n_frames) / original_fps
    target_n_frames = int(np.ceil(n_frames * (target_fps / original_fps)))
    target_times = np.arange(target_n_frames) / target_fps

    interpolator = interp1d(original_times, features, axis=0, kind='linear', fill_value='extrapolate')
    interpolated_features = interpolator(target_times)
    
    return interpolated_features


import numpy as np
from scipy.interpolate import interp1d

def interpolate_mat_rows(mat, original_fps=1.6, target_fps=6.0):
    n_rows, n_timepoints = mat.shape
    original_times = np.arange(n_timepoints) / original_fps
    target_n_timepoints = int(np.ceil(n_timepoints * (target_fps / original_fps)))
    target_times = np.arange(target_n_timepoints) / target_fps

    interpolated_mat = np.zeros((n_rows, len(target_times)))

    for i in range(n_rows):
        interpolator = interp1d(original_times, mat[i], kind='linear', fill_value='extrapolate')
        interpolated_mat[i] = interpolator(target_times)

    return interpolated_mat

# mat shape: (n_rows, n_timepoints), each row is a 1.6 fps time series
inferred_phase_high_fps = interpolate_mat_rows(inferred_phases_all_shifted, original_fps=1.6, target_fps=6.0)




new_indices = np.linspace(0, len(clean_values) - 1, int(len(clean_values) * (target_fps / original_fps)))
resampled_values = np.interp(new_indices, indices, clean_values)

In [ ]:
fig,ax_2 = plt.subplots(1,2)

ax2 = ax_2[1].twinx()
idxs=[0,-1]
transformed_angles = (270-np.degrees(np.concatenate(inferred_phases[[idxs]])+ np.pi))%360-180
behs = np.concatenate(initial_behaviors[[idxs]])
beh_map = [z_norm.min(), z_norm.min()/2 + z_norm.max()/2, z_norm.max()]
for i in range(3):
    ax2.hist(transformed_angles[np.abs(behs -beh_map[i]) < 0.01], bins=np.arange(-180,181,36), density=False, alpha=0.1, color=palette[i])
ax2.set_ylim(0,40)

ax_2[0].set_title(neuron)
colors = [plt.get_cmap("Purples")(i) for i in np.linspace(0, 1, 8)][1:]
colors[0] = (0,0,0,1)



times = onsets#this is time stim on ....innn seconds...?  
inferred_phases, final_behaviors, final_behaviors_all, inferred_phases_all, inferred_rad_all, behaviors_all, inferred_rad, initial_behaviors, initial_cts_beh, X_all_LSTM = analyze_data(resampled_vel, resampled_acc, resampled_curve, resampled_rev, resampled_turn, times, lag=8, inclusion_thresh = 3, model_type="ff", remove_revs=False)

for i in range(len(final_behs)):
    plot_rev_pro2((270-np.degrees(np.concatenate(worm_angles[0:5])+ np.pi +0*(np.random.rand(5*1589)-0.5)))%360-180, final_behs[i], np.concatenate(np.array(q_z[0:5])[:,5+T:]), ax=ax_2[0],  label=f"input: {input_str}", color=colors[i], window_size=np.pi/4, plot_type="line", num_bins=20)

if condition == "control":
    plot_rev_pro2((270-np.degrees(inferred_phases_all[:10000]+np.pi))%360-180, final_behaviors_all[:10000], ax=ax_2[1], color="k", window_size=np.pi/4, beh1=False, plot_type="box", num_bins=10)
    ax_2[1].set_title("control")
else:
    plot_results([0,-1], inferred_phases, final_behaviors, combined=True, ax=ax_2[1], colors=colors[:], num_bins=10, window_size=np.pi/4, plot_type="box", noise_level=0)
    ax_2[1].set_title(neuron)


ax_2[0].set_xlabel(r"$\theta$")
ax_2[1].set_xlabel(r"$\theta$")

fig.tight_layout()

In [ ]:

def plot_results(idxs, inferred_phases, final_behaviors, combined, ax, colors, window_size= np.pi/4, num_bins=6, noise_level=0, plot_type="box"):
    if combined == False:
        for i in idxs:
            plot_rev_pro2((270-np.degrees(inferred_phases[i] + np.pi+ noise_level*(-0.5+np.random.rand(inferred_phases[i].size))))%360-180, 
                          final_behaviors[i], 
                          ax=ax, color=colors[i+1], window_size=window_size, beh1=False, plot_type=plot_type, num_bins=num_bins)
    else:
        plot_rev_pro2((270-np.degrees(np.concatenate(inferred_phases[idxs]) + np.pi+ noise_level*(-0.5+np.random.rand(np.concatenate(inferred_phases[idxs]).size))))%360-180, np.concatenate(final_behaviors[idxs]), ax=ax, color=colors[-1], window_size=window_size, beh1=False, plot_type=plot_type, num_bins=num_bins)



In [ ]:
def plot_rev_pro2(phase, beh, z=[], label=None, ax=None, color=None, window_size=50, beh1=True, 
                  plot_type="box", num_bins=6):

    phase = np.array(phase)
    if beh1:
        beh = np.array(beh) == 1  # Convert to boolean

    idx = np.argsort(phase)
    sorted_phase = phase[idx]
    sorted_beh = beh[idx]

    # Compute smoothed behavioral response
    sorted_beh_mean, sorted_beh_std = circular_moving_success_stats(sorted_phase, sorted_beh, window_size=window_size)
    #sorted_beh_std = np.sqrt(sorted_beh_mean*(1-sorted_beh_mean)/len(idx))

    #sorted_beh_mean = sorted_beh

    if ax is None:
        fig, ax = plt.subplots()

    if plot_type == "line":
        # Line plot with shaded standard deviation
        l1 = ax.plot(sorted_phase, sorted_beh_mean, label=label, color=color)[0]
        ax.fill_between(sorted_phase, sorted_beh_mean - sorted_beh_std , 
                        sorted_beh_mean + sorted_beh_std , alpha=0.3, color=l1.get_color())

    elif plot_type == "box":
        # Bin phase data
        bin_edges = np.linspace(-180, 180, num_bins + 1)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        bin_indices = np.digitize(sorted_phase, bin_edges) - 1  # Assign each phase to a bin

        # Collect data in bins from the smoothed response
        binned_data = [sorted_beh_mean[bin_indices == i] for i in range(num_bins)]  
        
        # Remove empty bins
        valid_bins = [i for i, data in enumerate(binned_data) if len(data) > 0]
        bin_centers = bin_centers[valid_bins]
        binned_data = [binned_data[i] for i in valid_bins]
        print([(sum(~np.isnan(bd))) for bd in binned_data])

        # boot_data = []
        # for bd in binned_data:
        #     bin_boot = []
        #     for i in range(1000):
        #         n_samples = 4
        #         idx2 = np.arange(len(bd))
        #         np.random.shuffle(idx2)
        #         bin_boot.append(bd[idx2[:n_samples]].mean())
        #     boot_data.append(np.array(bin_boot))


        # Plot boxplot using the smoothed behavioral response
        ax.boxplot(binned_data, positions=bin_centers, widths=(360 / num_bins) * 0.7, patch_artist=True,
                   boxprops=dict(facecolor=color, alpha=0.7, edgecolor="k"), showfliers=False,
                   medianprops=dict(color="black"), manage_ticks=False)
        

    # Optional: Plot discrete states
    if np.size(z) > 0:
        sorted_z = z[idx]
        _, states, _ = bin_phase_data(sorted_phase, sorted_z)
        ax.imshow(np.array(states)[np.newaxis, :], extent=[-180, 180, -0.02, 0], cmap=cmap, vmin=0, vmax=6, aspect="auto")

    ax.set_xlabel("Phase")
    ax.set_ylabel("Rev. prob")
    ax.set_ylim(-0.02, 1.1)
    ax.set_xlim(-180, 180)

    if plot_type == "box":
        return ax, np.array([bin_i.mean() for bin_i in boot_data])
    else:
        return ax
    

def circular_moving_success_stats(theta, values, window_size):

    theta = np.radians(theta)
    smoothed_rates = np.zeros_like(values, dtype=float)
    smoothed_sem = np.zeros_like(values, dtype=float)

    for i, t in enumerate(theta):
        # Compute circular distances
        circular_diff = np.abs(np.angle(np.exp(1j * (theta - t))))  # Circular distance - handles that -pi and pi are the sam e
        mask = circular_diff <= window_size / 2  # Select points within window

        if np.any(mask):
            local_values = values[mask]
            smoothed_rates[i] = np.nanmean(local_values)  # Compute success rate (fraction of 1s)
            smoothed_sem[i] = scipy.stats.sem(local_values)
        else:
            smoothed_rates[i] = values[i]  # Default to original if no neighbors
            smoothed_sem[i] = 0  # No variance in a single point

    return smoothed_rates, smoothed_sem

